<a href="https://colab.research.google.com/github/sidt-ai/data-science-competitions/blob/main/dphi/ds74-smoker-status-prediction/notebooks/02_xgboost_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

In [ ]:
%%capture
!pip install xgboost==1.5.0
!pip install optuna==2.10.0

In [ ]:
import numpy as np
from scipy.stats import mode, ttest_ind
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.max_columns', None)

import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback

from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
xgb.__version__, optuna.__version__

('1.5.0', '2.10.0')

In [ ]:
SEED = 2311

os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [ ]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    GPU = True
except Exception:
    GPU = False

print(f'GPU available: {GPU}')

GPU available: True


# Data preprocessing

In [ ]:
train_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/dphi/ds74-smoker-status-prediction/data/raw/train_dataset.csv'
test_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/dphi/ds74-smoker-status-prediction/data/raw/test_dataset.csv'

In [ ]:
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

In [ ]:
features = test.columns.to_list()
TARGET = 'smoking'

[EDA notebook](https://github.com/sidt-ai/data-science-competitions/blob/main/dphi/ds74-smoker-status-prediction/notebooks/01_eda_and_baseline.ipynb)

In [ ]:
cat_features = ['hearing(left)', 'hearing(right)', 'dental caries']
reduced_features = [f for f in features if f not in ('hearing(left)', 'hearing(right)', 'Cholesterol', 'Urine protein')]

In [ ]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

### Creating folds

In [ ]:
N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
train['fold'] = -1

for fold, (_, idx) in enumerate(skf.split(X=train, y=train[TARGET])):
    train.loc[idx, 'fold'] = fold

# Hyperparameter tuning

In [ ]:
N_ESTIMATORS = 10000
EARLY_STOPPING_ROUNDS = 200
TREE_METHOD = 'gpu_hist' if GPU else 'hist'
OBJECTIVE = 'binary:logistic'
EVAL_METRIC = 'error'

In [ ]:
base_params = {
    'n_estimators': N_ESTIMATORS,
    'early_stopping_rounds': EARLY_STOPPING_ROUNDS,
    'tree_method': TREE_METHOD,
    'enable_categorical': GPU, #only available for gpu_hist
    'max_cat_to_onehot': 5,
    'eval_metric': EVAL_METRIC,
    'random_state': SEED,
    'verbosity': 0
}

In [ ]:
def objective(trial, base_params, data):
    #Defining hyperparameter search space
    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        
        # 'max_bin': trial.suggest_int('max_bin', 4, 512),
        
        'learning_rate': trial.suggest_float(
            'learning_rate', 0.05, 0.3, step=0.025),
        
        # 'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        
        'gamma': trial.suggest_float('gamma', 0.1, 20.0, step=0.1),
        
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 100),
        
        'max_delta_step': trial.suggest_float('max_delta_step', 1, 10, step=0.5),
        
        'subsample': trial.suggest_float('subsample', 0.5, 0.95, step=0.05),
        
        'colsample_bytree': trial.suggest_float(
            'colsample_bytree', 0.5, 0.95, step=0.05),
        
        'colsample_bylevel': trial.suggest_float(
            'colsample_bylevel', 0.5, 0.95, step=0.05),
        
        'colsample_bynode': trial.suggest_float(
            'colsample_bynode', 0.5, 0.95, step=0.05),
        
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1e3, log=True),
        
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1e3, log=True)
    }

    # if param_grid['booster'] == 'dart':
    #     param_grid['sample_type'] = 'weighted'
    #     param_grid['normalize_type'] = trial.suggest_categorical(
    #         'normalize_type', ['tree', 'forest'])
    #     param_grid['rate_drop'] = trial.suggest_float(
    #         'rate_drop', 0.1, 0.3)
    #     param_grid['skip_drop'] = trial.suggest_float(
    #         'skip_drop', 0.33, 0.67)

    model = xgb.XGBClassifier(
        **base_params, 
        **param_grid, 
        callbacks = [XGBoostPruningCallback(trial, 'validation_0-auc')])
    
    xtrain, xval, ytrain, yval = data
    
    model.fit(
        xtrain, ytrain,
        eval_set=[(xval, yval)],
        verbose=False)

    predictions = model.predict(xval)
    return f1_score(yval, predictions)

In [ ]:
def tune_hyperparameters(
        base_params,
        data,
        direction='maximize', 
        n_trials=5):
    
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        direction=direction)
    
    study.optimize(
        func=lambda trial: objective(trial, base_params, data),
        n_trials=n_trials,
        gc_after_trial=True)
    
    return study.best_params, study.best_value

### All features

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(train[features], 
                                              train[TARGET],
                                              test_size=0.2,
                                              stratify=train[TARGET],
                                              shuffle=True,
                                              random_state=SEED)

In [ ]:
%%capture
best_params, best_value = tune_hyperparameters(base_params=base_params,
                                               data=(xtrain, xval, ytrain, yval),
                                               direction='maximize',
                                               n_trials=50)

In [ ]:
print(f'Best f1_score: {best_value:.5f}')
print('Best params:')
for key, value in best_params.items():
    print(f'\t{key}: {value}')

Best f1_score: 0.71119
Best params:
	max_depth: 10
	learning_rate: 0.22500000000000003
	gamma: 0.1
	min_child_weight: 60
	max_delta_step: 10.0
	subsample: 0.75
	colsample_bytree: 0.8
	colsample_bylevel: 0.9
	colsample_bynode: 0.65
	reg_alpha: 1.2406159227159882e-05
	reg_lambda: 0.002124631422420931


In [ ]:
model_params_all = dict(base_params, **best_params)

### Reduced features

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(train[reduced_features], 
                                              train[TARGET],
                                              test_size=0.2,
                                              stratify=train[TARGET],
                                              shuffle=True,
                                              random_state=SEED)

In [ ]:
%%capture
best_params, best_value = tune_hyperparameters(base_params=base_params,
                                               data=(xtrain, xval, ytrain, yval),
                                               direction='maximize',
                                               n_trials=50)

In [ ]:
print(f'Best f1_score: {best_value:.5f}')
print('Best params:')
for key, value in best_params.items():
    print(f'\t{key}: {value}')

Best f1_score: 0.72046
Best params:
	max_depth: 14
	learning_rate: 0.3
	gamma: 0.2
	min_child_weight: 6
	max_delta_step: 9.5
	subsample: 0.6
	colsample_bytree: 0.8
	colsample_bylevel: 0.7
	colsample_bynode: 0.8
	reg_alpha: 1.3735876672882066
	reg_lambda: 1.8462370428126593


In [ ]:
model_params_red = dict(base_params, **best_params)

# Cross-validation

In [ ]:
def evaluate_model(train, test, features, model_params, n_splits=5):
    
    #out-of-fold predictions
    oof_proba = {} #probability
    oof_pred = {} #class

    test_proba = []
    test_pred = []
    cv_scores = []
    
    cv_start = time.time()
    for fold in range(n_splits):
        xtrain = train[train['fold'] != fold].reset_index(drop=True)
        ytrain = xtrain[TARGET]

        xval = train[train['fold'] == fold].reset_index(drop=True)
        yval = xval[TARGET]
        val_idx = xval.index.to_list()

        fold_start = time.time()

        model = xgb.XGBClassifier(**model_params)
        
        model.fit(
            xtrain[features], ytrain,
            eval_set=[(xval[features], yval)], 
            verbose=False)

        val_pred = model.predict(xval[features])
        oof_pred.update(dict(zip(val_idx, val_pred)))
        val_proba = model.predict_proba(xval[features])[:, 1]
        oof_proba.update(dict(zip(val_idx, val_proba)))        

        score = f1_score(yval, val_pred)
        cv_scores.append(score)

        fold_end = time.time()

        print(f'Fold #{fold}: f1_score = {score:.5f} \
        [Time: {fold_end - fold_start:.2f}s]')
        
        test_pred.append(model.predict(test[features]))
        test_proba.append(model.predict_proba(test[features])[:, 1])
        
    cv_end = time.time()

    print(f'Average f1-score = {np.mean(cv_scores):.5f} \
    with std. dev. = {np.std(cv_scores):.5f}')
    print(f'[Total time: {cv_end - cv_start:.2f}s]')

    oof_pred = pd.DataFrame.from_dict(oof_pred, orient='index').reset_index()
    oof_proba = pd.DataFrame.from_dict(oof_proba, orient='index').reset_index()
    
    test_pred = mode(np.column_stack(test_pred), axis=1).mode
    test_proba = np.mean(np.column_stack(test_proba), axis=1)
    
    return oof_pred, oof_proba, test_pred, test_proba

### All features

In [ ]:
oof_pred, oof_proba, test_pred, test_proba = evaluate_model(train, test, 
                                                            features, 
                                                            model_params_all)

Fold #0: f1_score = 0.70584         [Time: 27.71s]
Fold #1: f1_score = 0.70384         [Time: 27.66s]
Fold #2: f1_score = 0.70060         [Time: 27.85s]
Fold #3: f1_score = 0.71007         [Time: 27.88s]
Fold #4: f1_score = 0.71027         [Time: 27.81s]
Average f1-score = 0.70613     with std. dev. = 0.00370
[Total time: 177.26s]


In [ ]:
sub1 = pd.DataFrame({'smoking': test_pred.ravel()})
sub1.to_csv('sub-02-full-features.csv', index=False)

In [ ]:
!head sub-02-full-features.csv

smoking
1
1
0
0
0
1
0
1
0


### Reduced features

In [ ]:
oof_pred, oof_proba, test_pred, test_proba = evaluate_model(train, test, 
                                                            reduced_features, 
                                                            model_params_red)

Fold #0: f1_score = 0.71599         [Time: 20.85s]
Fold #1: f1_score = 0.70681         [Time: 20.03s]
Fold #2: f1_score = 0.70406         [Time: 20.60s]
Fold #3: f1_score = 0.71424         [Time: 20.58s]
Fold #4: f1_score = 0.71406         [Time: 20.51s]
Average f1-score = 0.71103     with std. dev. = 0.00470
[Total time: 118.35s]


In [ ]:
sub2 = pd.DataFrame({'smoking': test_pred.ravel()})
sub2.to_csv('sub-02-reduced-features.csv', index=False)

In [ ]:
!head sub-02-reduced-features.csv

smoking
1
1
0
0
0
1
0
1
0
